In [ ]:
!git clone https://github.com/pjreddie/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 5913, done.
remote: Total 5913 (delta 0), reused 0 (delta 0), pack-reused 5913
Receiving objects: 100% (5913/5913), 6.34 MiB | 20.60 MiB/s, done.
Resolving deltas: 100% (3917/3917), done.


In [ ]:
cd darknet

/content/darknet


In [ ]:
!make

mkdir -p obj
mkdir -p backup
mkdir -p results
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/gemm.c -o obj/gemm.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/utils.c -o obj/utils.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/cuda.c -o obj/cuda.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/deconvolutional_layer.c -o obj/deconvolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/convolutional_layer.c -o obj/convolutional_layer.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/list.c -o obj/list.o
gcc -Iinclude/ -Isrc/ -Wall -Wno-unused-result -Wno-unknown-pragmas -Wfatal-errors -fPIC -Ofast -c ./src/image.c -o obj/image.o
gcc 

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights

--2020-08-18 06:58:00--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M   342KB/s    in 10m 9s  

2020-08-18 07:08:10 (398 KB/s) - ‘yolov3.weights’ saved [248007048/248007048]



In [ ]:
!./darknet detect cfg/yolov3.cfg yolov3.weights data/dog.jpg

layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

In [ ]:
!./darknet detector test cfg/coco.data cfg/yolov3.cfg yolov3.weights data/dog.jpg


layer     filters    size              input                output
    0 conv     32  3 x 3 / 1   608 x 608 x   3   ->   608 x 608 x  32  0.639 BFLOPs
    1 conv     64  3 x 3 / 2   608 x 608 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    2 conv     32  1 x 1 / 1   304 x 304 x  64   ->   304 x 304 x  32  0.379 BFLOPs
    3 conv     64  3 x 3 / 1   304 x 304 x  32   ->   304 x 304 x  64  3.407 BFLOPs
    4 res    1                 304 x 304 x  64   ->   304 x 304 x  64
    5 conv    128  3 x 3 / 2   304 x 304 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    6 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
    7 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
    8 res    5                 152 x 152 x 128   ->   152 x 152 x 128
    9 conv     64  1 x 1 / 1   152 x 152 x 128   ->   152 x 152 x  64  0.379 BFLOPs
   10 conv    128  3 x 3 / 1   152 x 152 x  64   ->   152 x 152 x 128  3.407 BFLOPs
   11 res    8                 152 x 

In [ ]:
import numpy as np
import cv2

confidenceThreshold = 0.5
NMSThreshold = 0.3

modelConfiguration = '/content/darknet/cfg/yolov3.cfg'
modelWeights = '/content/darknet/yolov3.weights'
inputimage = '/content/darknet/data/dog.jpg'
labelsPath = '/content/darknet/data/coco.names'

In [ ]:
labels = open(labelsPath).read().strip().split('\n')
labels

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [ ]:
np.random.seed(10)
COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype="uint8")
COLORS

array([[  9,  69, 117],
       [197, 125, 156],
       [123,  76, 228],
       [ 79,   5,  15],
       [114, 157, 126],
       [ 64, 197,  54],
       [162, 113, 109],
       [105, 113, 123],
       [156, 177, 191],
       [156,  36, 248],
       [212, 217,  39],
       [158, 127, 221],
       [146,  84, 149],
       [240,  69, 140],
       [ 57, 157, 167],
       [113,   6, 113],
       [ 63, 180,  50],
       [250, 167, 142],
       [181,   8,  36],
       [178, 194,  73],
       [ 36, 252,  67],
       [  0, 217,  74],
       [ 43, 234, 121],
       [123,  67,  40],
       [112, 157,  22],
       [246,  41, 127],
       [ 38, 164, 189],
       [115, 175, 115],
       [185,  14, 175],
       [ 16, 150,  17],
       [244, 100, 209],
       [ 12, 209, 239],
       [192,   2,   1],
       [139, 201,   8],
       [ 86,  54,   8],
       [ 31, 131,  88],
       [141,  12, 228],
       [107, 237,   7],
       [208, 122,  28],
       [184,  50,  62],
       [130, 206, 156],
       [ 33, 126

In [ ]:
net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
image = cv2.imread(inputimage)
(H, W) = image.shape[:2]

net, image, (H, W)

(<dnn_Net 0x7f571eda3090>, array([[[ 50,  58,  57],
         [ 51,  59,  58],
         [ 53,  61,  60],
         ...,
         [ 47,  89, 142],
         [ 41,  50,  88],
         [ 47,  71,  63]],
 
        [[ 51,  59,  58],
         [ 51,  59,  58],
         [ 52,  60,  59],
         ...,
         [ 37,  74, 124],
         [ 41,  50,  83],
         [ 46,  70,  58]],
 
        [[ 51,  59,  58],
         [ 51,  59,  58],
         [ 52,  60,  59],
         ...,
         [ 25,  54,  98],
         [ 48,  54,  77],
         [ 43,  62,  45]],
 
        ...,
 
        [[179, 168, 160],
         [179, 168, 160],
         [182, 171, 163],
         ...,
         [ 64,  64,  80],
         [ 36,  39,  53],
         [ 48,  53,  62]],
 
        [[180, 169, 161],
         [179, 168, 160],
         [175, 164, 156],
         ...,
         [ 61,  61,  79],
         [ 37,  40,  54],
         [ 52,  56,  67]],
 
        [[176, 165, 157],
         [178, 167, 159],
         [172, 161, 153],
         ...,
  

In [ ]:
layerName = net.getLayerNames()
layerName

['conv_0',
 'bn_0',
 'relu_0',
 'conv_1',
 'bn_1',
 'relu_1',
 'conv_2',
 'bn_2',
 'relu_2',
 'conv_3',
 'bn_3',
 'relu_3',
 'shortcut_4',
 'conv_5',
 'bn_5',
 'relu_5',
 'conv_6',
 'bn_6',
 'relu_6',
 'conv_7',
 'bn_7',
 'relu_7',
 'shortcut_8',
 'conv_9',
 'bn_9',
 'relu_9',
 'conv_10',
 'bn_10',
 'relu_10',
 'shortcut_11',
 'conv_12',
 'bn_12',
 'relu_12',
 'conv_13',
 'bn_13',
 'relu_13',
 'conv_14',
 'bn_14',
 'relu_14',
 'shortcut_15',
 'conv_16',
 'bn_16',
 'relu_16',
 'conv_17',
 'bn_17',
 'relu_17',
 'shortcut_18',
 'conv_19',
 'bn_19',
 'relu_19',
 'conv_20',
 'bn_20',
 'relu_20',
 'shortcut_21',
 'conv_22',
 'bn_22',
 'relu_22',
 'conv_23',
 'bn_23',
 'relu_23',
 'shortcut_24',
 'conv_25',
 'bn_25',
 'relu_25',
 'conv_26',
 'bn_26',
 'relu_26',
 'shortcut_27',
 'conv_28',
 'bn_28',
 'relu_28',
 'conv_29',
 'bn_29',
 'relu_29',
 'shortcut_30',
 'conv_31',
 'bn_31',
 'relu_31',
 'conv_32',
 'bn_32',
 'relu_32',
 'shortcut_33',
 'conv_34',
 'bn_34',
 'relu_34',
 'conv_35',
 'bn

In [ ]:
layerName = [layerName[i[0] - 1] for i in net.getUnconnectedOutLayers()]
layerName

['yolo_82', 'yolo_94', 'yolo_106']

In [ ]:
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB = True, crop = False)
nb = net.setInput(blob)
layersOutputs = net.forward(layerName)

boxes = []
confidences = []
classIDs = []

blob, nb, layersOutputs

(array([[[[0.22352943, 0.23529413, 0.2392157 , ..., 0.76470596,
           0.6117647 , 0.28235295],
          [0.227451  , 0.23137257, 0.2392157 , ..., 0.78823537,
           0.5137255 , 0.24705884],
          [0.23137257, 0.23137257, 0.2392157 , ..., 0.79215693,
           0.3803922 , 0.22352943],
          ...,
          [0.61960787, 0.6392157 , 0.6156863 , ..., 0.50980395,
           0.40784317, 0.24313727],
          [0.627451  , 0.62352943, 0.6039216 , ..., 0.5137255 ,
           0.3529412 , 0.23529413],
          [0.61960787, 0.6       , 0.6117647 , ..., 0.5019608 ,
           0.3529412 , 0.20392159]],
 
         [[0.227451  , 0.2392157 , 0.24705884, ..., 0.9294118 ,
           0.48235297, 0.24313727],
          [0.23137257, 0.23529413, 0.24313727, ..., 0.9294118 ,
           0.3921569 , 0.23529413],
          [0.23529413, 0.23529413, 0.24313727, ..., 0.8980393 ,
           0.27450982, 0.24313727],
          ...,
          [0.6509804 , 0.67058825, 0.64705884, ..., 0.4784314 ,
   

In [ ]:
for output in layersOutputs:
    for detection in output:
        scores = detection[5:]
        classID = np.argmax(scores)
        confidence = scores[classID]
        if confidence > confidenceThreshold:
            box = detection[0:4] * np.array([W, H, W, H])
            (centerX, centerY,  width, height) = box.astype('int')
            x = int(centerX - (width/2))
            y = int(centerY - (height/2))

            boxes.append([x, y, int(width), int(height)])
            confidences.append(float(confidence))
            classIDs.append(classID)


boxes, confidences, classIDs

([[472, 86, 220, 79], [117, 124, 452, 307], [122, 223, 197, 320]],
 [0.9396677613258362, 0.9898989796638489, 0.9979043006896973],
 [7, 1, 16])

In [ ]:
detectionNMS = cv2.dnn.NMSBoxes(boxes, confidences, confidenceThreshold, NMSThreshold)
detectionNMS, 'length of detectionNMS = ', len(detectionNMS)

(array([[2],
        [1],
        [0]], dtype=int32), 'length of detectionNMS = ', 3)

In [ ]:
if(len(detectionNMS) > 0):
    for i in detectionNMS.flatten():
        (x, y) = (boxes[i][0], boxes[i][1])
        (w, h) = (boxes[i][2], boxes[i][3])
        
        detected = labels[classIDs[i]]
        print(detected)
        
        color = [int(c) for c in COLORS[classIDs[i]]]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        text = '{}: {:.4f}'.format(labels[classIDs[i]], confidences[i])
        cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


dog
bicycle
truck
